In [21]:
%pip install torch
%pip install numpy
import random
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [25]:
class Bride:
    def __init__(self, n):
        self.n = n  # количество претендентов
        self.candidates = list(range(n))  # список кандидатов
        random.shuffle(self.candidates)  # Перемешиваем кандидатов

    def select(self, strategy):
        best_seen = -1 # лучший кандидат

        for i in range(self.n):
            candidate = self.candidates[i]
            if candidate > best_seen:
                best_seen = candidate
                if strategy(i):
                    return best_seen  

        return None

def stopping_rule(current_index):
    return current_index > int(0.37 * N)  # 1/e = 0.37


def run_simulation(trials, n):
    successful_selections = 0

    for _ in range(trials):
        bride = Bride(n)

        if bride.select(stopping_rule) == max(list(range(n))):
            successful_selections += 1

    return successful_selections / trials

N = 1000  # количество претендентов
trials = 1000  # количество симуляций
success_rate = run_simulation(trials, N)

print(f"Вероятность успешного выбора лучшего кандидата: {success_rate:.2f}")

Вероятность успешного выбора лучшего кандидата: 0.38


In [ ]:
import random
import torch
import torch.nn as nn
import torch.optim as optim

N = 100         
trials = 10000  
learning_rate = 0.001
epochs = 5
batch_size = 64

def generate_dataset(n, strategy_function):
    data = []
    labels = []
    
    for _ in range(trials):
        candidates = list(range(n))
        random.shuffle(candidates)
        
        best_seen = -1
        decisions = []
        
        noise_level = 0.01
        for i in range(n):
            candidate = (candidates[i] / (n - 1)) + random.uniform(-noise_level, noise_level)
            best_seen_norm = ((best_seen / (n - 1)) + random.uniform(-noise_level, noise_level)) if best_seen != -1 else 0
            
            if candidates[i] > best_seen:
                best_seen = candidates[i]
            
            # Применение стратегии
            accept = strategy_function(i)
            decisions.append((candidate, best_seen_norm))
            labels.append(int(accept and candidates[i] == max(candidates)))
        
        data.extend(decisions)
    
    return data, labels

# Стратегия остановки
def stopping_rule(current_index):
    return current_index > int(0.37 * N)  # 1/e ≈ 0.37

# Нейронная сеть
class DecisionNet(nn.Module):
    def __init__(self):
        super(DecisionNet, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Подготовка данных
data, labels = generate_dataset(N, stopping_rule)
data = torch.tensor(data, dtype=torch.float32)
labels = torch.tensor(labels, dtype=torch.float32).view(-1, 1)

# Создание модели
model = DecisionNet()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
dataset = torch.utils.data.TensorDataset(data, labels)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

for epoch in range(epochs):
    total_loss = 0.0
    batch_count = 0

    for inputs, target in dataloader:
        optimizer.zero_grad()
        output = model(inputs)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        batch_count += 1

    # Средняя ошибка за эпоху
    average_loss = total_loss / batch_count
    print(f"Эпоха [{epoch + 1}/{epochs}], Средняя потеря: {average_loss:.4f}")

# Функция оценки
def evaluate_model(model, n, trials):
    successful_selections = 0
    
    for _ in range(trials):
        candidates = list(range(n))
        random.shuffle(candidates)
        
        best_seen = -2
        noise_level = 0.01
        for i in range(n):
            candidate = (candidates[i] / (n - 1)) + random.uniform(-noise_level, noise_level)
            best_seen_norm = ((best_seen / (n - 1)) + random.uniform(-noise_level, noise_level)) if best_seen != -2 else 0
            input_data = torch.tensor([[candidate, best_seen_norm]], dtype=torch.float32)
            
            with torch.no_grad():
                decision = model(input_data).item() > 0.5
            
            if candidates[i] > best_seen:
                best_seen = candidates[i]
                if decision:
                    if candidates[i] == max(candidates):
                        successful_selections += 1
                    break
    
    return successful_selections / trials

# Запуск оценки
success_rate_nn = evaluate_model(model, N, trials)
print(f"Вероятность успешного выбора с помощью нейронной сети: {success_rate_nn:.2f}")
